In [3]:
import pandas as pd
from model import BMI_Model
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import scipy.io as sio
import math

In [4]:
constant = {}
constant['Ix'] = 200
constant['Vx'] = 10
constant['Br'] = 0.1
constant['Bu'] = 0.01
constant['row'] = 0.04
constant['Thetax'] = 0.5
constant['etax'] = 0.7
constant['lamda1x'] = 150
constant['lamda2x'] = 10
constant['Gammax'] = 0.001
constant['Cx'] = 25
constant['epsx'] = 0.05
constant['xix'] = 4
constant['hx'] = 0.01
constant['TI'] = 0.7
constant['Eix'] = 0
constant['delta'] = 0.1
constant['v'] = 0.15
initalVal = {}
initalVal['x1'] = 0.5
initalVal['x2'] = 0.5
initalVal['y1'] = 0.5
initalVal['y2'] = 0.5
initalVal['g1'] = 0
initalVal['p1'] = 0.5
initalVal['dp1'] = 0
initalVal['p2'] = 0.5
initalVal['dp2'] = 0
initalVal['q1'] = 0
initalVal['q2'] = 0
initalVal['f1'] = 0
initalVal['f2'] = 0
initalVal['s1_1'] = 0
initalVal['s1_2'] = 0
initalVal['s2_1'] = 0
initalVal['s2_2'] = 0
initalVal['c1'] = 0
initalVal['c2'] = 0
initalVal['y0'] = 0
initalVal['vObj'] = -0.001
initalVal['Z'] = 3
initalVal['g0'] = np.random.normal(0.75, 0.05)

In [6]:
# trainList = ['U1', 'U2', 'Y1', 'Y2', 'A1', 'A2', 'Y0', 'P1', 'DP1', 'TI', 'R1', 'R2','DP2', 'X1', 'X2']
trainList = ['U1', 'U2', 'Y1', 'Y2', 'A1', 'A2', 'Y0', 'P1', 'DP1', 'TI']
dataAll = {}
for key in trainList:
    dataAll[key] = []
#-------------old Row Data--------------#
for i in range(1600):
    initalVal['g0'] = np.random.normal(0.75, 0.05)
    model = BMI_Model(constant, initalVal)
    model.ns = 146
#     dataTemp = model.modelRun() # original model
    dataTemp = model.modelRunDynamic() # improved model
    for key in dataAll.keys():
        dataAll[key].extend(dataTemp[key])
print(len(dataAll['Y0']))
dataFrame = pd.DataFrame(dataAll)
dataFrame.to_csv('data_python_dynamic.csv')
dataFrame.head()

233600


,A1,A2,DP1,P1,TI,U1,U2,Y0,Y1,Y2
0,0.5,0.5,0.0,0.5,0.700,0.01,0.01,0.0,0.5,0.5
1,0.5,0.5,0.0,0.5,0.699,0.01,0.01,0.0,0.5,0.5
2,0.5,0.5,0.0,0.5,0.698,0.01,0.01,0.0,0.5,0.5
3,0.5,0.5,0.0,0.5,0.697,0.01,0.01,0.0,0.5,0.5
4,0.5,0.5,0.0,0.5,0.696,0.01,0.01,0.0,0.5,0.5


In [7]:
T2 = dataFrame['Y0']
T_new = []
for i in range(1600):
    tmp = [0] * 10
    tmp.extend(T2[i * 146: (i + 1)* 146 - 1])
    for j in range(len(tmp) - 9):
        T_new.append(tmp[j: j + 10])
T_new = np.array(T_new)
np.save('python_train_data_Y0.csv', T_new)

In [17]:
u1 = dataFrame['U1']
u2 = dataFrame['U2']
y1 = dataFrame['Y1']
y2 = dataFrame['Y2']
a1 = dataFrame['A1']
a2 = dataFrame['A2']
train_data = []
target = []
for i in range(1600):
    tmp = np.array([0] * 9)
    u1_tmp = np.hstack((tmp, u1[i * 146: (i + 1)* 146]))
    u2_tmp = np.hstack((tmp, u2[i * 146: (i + 1)* 146]))
    y1_tmp = np.hstack((tmp, y1[i * 146: (i + 1)* 146]))
    y2_tmp = np.hstack((tmp, y2[i * 146: (i + 1)* 146]))
    a1_tmp = np.hstack((tmp, a1[i * 146: (i + 1)* 146]))
    a2_tmp = np.hstack((tmp, a2[i * 146: (i + 1)* 146]))
    target.extend(T2[i * 146 + 1: (i + 1)* 146])
    for j in range(len(u1_tmp) - 10):
        train_data.append(np.hstack((u1_tmp[j: j + 10], u2_tmp[j: j + 10],
                                     y1_tmp[j: j + 10], y2_tmp[j: j + 10],
                                     a1_tmp[j: j + 10], a2_tmp[j: j + 10])))
train_data = np.array(train_data)
np.save('python_train_data_dynamic.csv', train_data)
target = np.array(target)
np.save('python_train_target_data_dynamic.csv', target)

In [16]:
print(target.shape)

(232000,)
